### Case - [Pierre-Antoine Denarié]

### TL;DR
Objective: predict the time it takes for a bus to travel from first to final stop, at any given point in the future.

Points to work out:
- correlation between checkin data & scheduling ties length/delta in prediction --> create feature delta time
- Join checkin data with scheduling data --> PCA
- correlation between stoparea code & join of number of checkins during said Journey

Plots:
- plot checkins over time

features to create:
- split checkin data
- feature delta time


### Housekeeping & Imports

In [1]:
# Clear all variables from the workspace
%reset -f

In [2]:
### Package Imports
# System
from __future__ import annotations
import os
from pathlib import Path
import re
import logging
from typing import Dict
from functools import partial

# Data
import numpy as np
import pandas as pd

# Own packages
from src.myfunctions import *

# Import visualisation libraries
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

# Pretty display for notebooks
%matplotlib inline

# Setup Logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")


INFO: Pandas backend loaded 2.3.1
INFO: Numpy backend loaded 2.1.3
INFO: Pyspark backend NOT loaded
INFO: Python backend loaded


In [3]:
# Parameter
DATA_DIR = "Data"
DATA_RAW_DIR = DATA_DIR + "/Raw"
PROCESSED_DIR = DATA_DIR + "/Processed"

REPORTS : str = "Reports"
PROFILING : str = REPORTS + "/Profiling"

### Data Loading

In [4]:
# Clean Data imports
dfs = load_csv_folder(DATA_RAW_DIR)
print(dfs.keys())

INFO: Loaded bus_trips from bus_trips.csv | shape=(591766, 11)
INFO: Loaded check_ins from check_ins.csv | shape=(41923, 2)
INFO: Loaded stops from stops.csv | shape=(5636, 3)


dict_keys(['bus_trips', 'check_ins', 'stops'])


### Data exploration

In [5]:
# Generate profling reports
df_profiles = generate_profiling_reports(dfs, title="Data Profiling Report")

INFO: Generated profiling report for bus_trips
INFO: Generated profiling report for check_ins
INFO: Generated profiling report for stops


In [6]:
# View profiling reports
display_profiling_reports_web(profiles=df_profiles)

INFO: Opened profiling report for bus_trips
INFO: Opened profiling report for check_ins
INFO: Opened profiling report for stops


In [10]:
# Display time series data
pipeline = {
    "check_ins": [
        Step("add_feature", when=has_cols("id"), fn=partial(add_feature, src="id", dest='id_Datetime', function=parse_time_column))
    ]
}

dfs_processed = run_pipeline(dfs, pipeline)
display_profiling_reports_web(generate_profiling_reports(dfs_processed, title="Data Profiling Processed Report", timeseries=True, sortby="id_Datetime"), refresh_results=True)


INFO: ▶ bus_trips: 0 steps
INFO: ▶ check_ins: 1 steps
INFO:   - add_feature: (41923, 2) -> (41923, 3) (228 ms)
INFO: ▶ stops: 0 steps
ERROR: Error generating profiling report for bus_trips: 'id_Datetime'
INFO: Generated profiling report for check_ins
ERROR: Error generating profiling report for stops: 'id_Datetime'
100%|██████████| 3/3 [00:19<00:00,  6.65s/it]
INFO: Saved profiling report for check_ins
INFO: Opened profiling report for check_ins


In [ ]:
def inspect_duplicate_rows(df: pd.DataFrame, example_index: int) -> pd.DataFrame:
    """
    Inspect duplicate rows in a DataFrame by a single example.

    Args:
        df (pd.DataFrame): The DataFrame to inspect.
        example_index (int): The index of the example row to match against.

    Returns:
        pd.DataFrame: A DataFrame containing the duplicate rows.
    """
    return df[(df == df.iloc[example_index]).all(axis=1)]


def count_duplicate_rows(df: pd.DataFrame) -> pd.Series:
    """
    Count duplicate rows in a DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to inspect.

    Returns:
        pd.Series: A Series containing the count of duplicate rows.
    """

    df_copy: pd.DataFrame = df.copy()
    df_copy['__duplicate_count__'] = 0

    while df_copy.shape[0] > 0:
        # Duplicates
        duplicates_indices = (df_copy == df_copy.iloc[0]).all(axis=1)
        duplicate_count = duplicates_indices.sum()

        df_copy.loc[duplicates_indices, '__duplicate_count__'] = duplicate_count

        # Remove the inspected duplicate rows from the DataFrame
        df_copy = df_copy[~duplicates_indices]

    return df_copy['__duplicate_count__']


In [59]:
count_duplicate_rows(dfs['bus_trips'])

0         4
1         4
2         4
3         4
4         4
         ..
591761    2
591762    2
591763    2
591764    2
591765    2
Length: 591766, dtype: int64

In [83]:
from typing import Sequence

def duplicate_count_per_row(
    df: pd.DataFrame,
    subset: Sequence[str] | None = None,
    *,
    treat_na_as_equal: bool = True,
    return_series: bool = True
) -> pd.DataFrame|pd.Series:
    """
    For each original row, return the size of its duplicate group.

    Returns a Series aligned with `df.index`.
    """
    cols = list(df.columns) if subset is None else list(subset)

    if treat_na_as_equal:
        sizes = (df
                 .groupby(cols, dropna=False)
                 .size()
                 .rename("__dup_count__")
                 .reset_index())
        if return_series:
            result = df[cols].merge(sizes, on=cols, how="left")["__dup_count__"]
        else:
            result = df[cols].merge(sizes, on=cols, how="left")
    else:
        mask = df[cols].notna().all(axis=1)
        sizes = (df.loc[mask, cols]
                 .groupby(cols, dropna=False)
                 .size()
                 .rename("__dup_count__")
                 .reset_index())
        if return_series:
            result = df[cols].merge(sizes, on=cols, how="left")["__dup_count__"].fillna(1)
        else:
            result = df[cols].merge(sizes, on=cols, how="left")
    result.index = df.index
    result.name = "duplicate_count"
    return result

In [86]:
# 2) Keep all rows but know how many identical rows each belongs to
duplicate_count_per_row(dfs['bus_trips'], return_series=True).value_counts()

duplicate_count
2    308080
4    283672
1        14
Name: count, dtype: int64

In [87]:
## closer inspection of duplicate rows by a single example
inspect_duplicate_rows(dfs['bus_trips'], 3122)

,lineplanningnumber,journeynumber,vehiclenumber,userstopcode_start,userstopcode_end,messagetype_begin,messagetype_end,operatingday,departure_time,realized_time,planned_time
3122,1,1005,5149,54261502,53490410,DEPARTURE,ARRIVAL,2021-09-08,2021-09-08 07:07:24,00:32:54,00:32:58
74041,1,1005,5149,54261502,53490410,DEPARTURE,ARRIVAL,2021-09-08,2021-09-08 07:07:24,00:32:54,00:32:58
144960,1,1005,5149,54261502,53490410,DEPARTURE,ARRIVAL,2021-09-08,2021-09-08 07:07:24,00:32:54,00:32:58
215879,1,1005,5149,54261502,53490410,DEPARTURE,ARRIVAL,2021-09-08,2021-09-08 07:07:24,00:32:54,00:32:58


Seems many unique rows have 3 duplicates within the data set of bus_trips.csv. Lets see how many exactly

### Data Cleaning

bus_trips:
- remove duplicates

stops:
- address missing values

ceck_ins:

### Feature Engineering

### Data Staging